In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
connect_4 = fetch_ucirepo(id=26) 
  
# data (as pandas dataframes) 
X = connect_4.data.features 
y = connect_4.data.targets 

In [2]:
import time
import numpy as np
import pandas as pd
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [3]:
'''
To begin processing the data, we needed to translate the values of x/o/b (game pieces or (b)lank/empty spots)
and win/loss/draw into numbers that our SVMs could better work with. Once done with that, they are transformed
into arrays for easy processing because out of the box SVMs are not set up to work with pandas dataframes.
'''

X_encode_map = {'x':1, 'o': 2, 'b': 0}
X_encoded_dataframes = X.replace(X_encode_map)
X_encoded = X_encoded_dataframes.values

y_encode_map = {"win": 1, "loss": 2, "draw": 0}
y_encoded_dataframes = y.replace({"class": y_encode_map})
y_encoded = y_encoded_dataframes['class'].values

'''
Due to the size of the data (approximately 67.5k instances), we quickly realized we would need to find a way to speed
up data processing if we wanted to do a thorough investigation of different hyperparameter settings without spending
hours on every configuration. We will discuss this in more detail in several relevant sections below. To start,
the code cell below this one has the PCA code necessary to determine the amount of principle components we can reduce
to while still maintaing the majority of our data's information. The lines of code below this comment utilize the PCA
findings in combination with an additional train_test_split that discards a portion of our data to give us vastly
improved testing times to determine which hyperparameters are even reasonable to investigate more deeply.
'''

# X_encoded_reduced, X_discard, y_encoded_reduced, y_discard = train_test_split(
#     X_encoded, y_encoded, test_size= 1/2, random_state=5)

# scaler = StandardScaler()
# X_std_encoded_reduced = scaler.fit_transform(X_encoded_reduced)

# #X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=5)
# X_train, X_test, y_train, y_test = train_test_split(
#     X_std_encoded_reduced, y_encoded_reduced, test_size=0.3, random_state=5)

# pca = PCA(n_components=24).fit(X_train)
# X_train_pca = pca.transform(X_train)
# X_test_pca = pca.transform(X_test)

#No scaling necessary due to the nature of the data??
#TEST SIZE THE LAST VALUE TO EXPERIMENT WITH - .4 feels overboard. Feels like as low as 10% could work.
#Keeping a set of test data out throughout the process to only use at the end?
#Kfold of 3 actually very slightly reduced accuracy - theory: could be due to closely related models (say only one or two moves difference) being split across the folds.
#   These closely related models could be crucial for helping the SVM to understand what particular small differences can have on the outcome.
#Kfold - Taking a test set out at the very beginning then testing with that after cross validation?

'''
The few lines of code below are what we used when we wanted to test hyperparameters with the full dataset.
'''

X_train_pca, X_test_pca, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.3, random_state=5)



In [ ]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_encoded)
# pca = PCA(n_components=42)
# X_pca = pca.fit_transform(X_scaled)

# explained_variance_ratio = pca.explained_variance_ratio_

# # Initialize variables to store feature counts for desired explained variance thresholds
# feature_counts = {'80%': None, '85%': None, '90%': None}
# total_variance = 0

# # Calculate the cumulative explained variance ratio
# for i, ratio in enumerate(explained_variance_ratio):
#     total_variance += ratio
#     for threshold in feature_counts:
#         if feature_counts[threshold] is None and total_variance >= float(threshold.strip('%')) / 100:
#             feature_counts[threshold] = i + 1  # Add 1 to convert from 0-based index to count of features
#             break

# # Print feature counts for different thresholds
# print("Number of features needed to maintain:")
# for threshold, count in feature_counts.items():
#     print(f"{threshold} of the original information:", count)



# print("Explained variance by each component:", pca.explained_variance_ratio_)
# plt.figure(figsize=(8, 4))
# plt.bar(range(1, 43), pca.explained_variance_ratio_, alpha=0.5, align='center', label='Individual explained variance')
# plt.step(range(1, 43), np.cumsum(pca.explained_variance_ratio_), where='mid', label='Cumulative explained variance')
# plt.ylabel('Explained variance ratio')
# plt.xlabel('Principal components')
# plt.legend(loc='best')
# plt.tight_layout()

# components_df = pd.DataFrame(pca.components_, columns=X_encoded_dataframes.columns, index=['PC-' + str(i) for i in range(1, 43)])
# print(components_df)

# plt.figure(figsize=(12, 6))
# avg_feature_contributions = components_df.iloc[:42].mean()  # Calculate the average feature contributions for the first 21 principal components
# avg_feature_contributions.plot(kind='bar')
# plt.title('Average feature contributions for the first 42 principal components')
# plt.xlabel('Original features')
# plt.ylabel('Average feature contribution')
# plt.show()


In [48]:
# accuracies_linear = []
#RENAME TO ACCURACY UNLESS YOU START AVERAGING ACCURACIES
#models = ['linear', ]

# model_linear = svm.SVC(kernel='linear')
# model_linear.fit(X_train, y_train)
# y_pred_linear = model_linear.predict(X_test)

#Linear ultimately not a good choice for a classification with 3 classes. Could mitigate these factors by setting up a
#One versus Rest model or train the model for every pair of classes compared to one another but our project is
#More focused on comparing SVM to NN so we opted for models that can handle the complexity of C4
#right out of the box.




# the n_samples and n_features below are still a bit unclear to me, they just make an error go away
# accuracies_linear = []
n_samples, n_features = X_train.shape
# svc_linear = LinearSVC(random_state=5, dual=n_samples > n_features)
# svc_linear.fit(X_train, y_train)
# y_pred_linear = svc_linear.predict(X_test)
# accuracies_linear.append(np.mean(y_pred_linear == y_test))

# print(accuracies_linear)

#bad linear accuracy is 0.65674955595 and consistently takes over a minute to compute
#good linear accuracy is 0.65714426682 and consistently takes about 5 seconds.
#This may be because svm.SVC is using a more generalized library whereas the LinearSVC is
#specifically for linear SVCs and therefore is likely highly optimized for those scenarios.



n_samples, n_features = X_train_pca.shape  # Notice we use the transformed data shape here
svc_linear = LinearSVC(random_state=5, dual=n_samples > n_features)
svc_linear.fit(X_train_pca, y_train)

# Predict using the trained LinearSVC on the PCA-transformed test data
y_pred_linear = svc_linear.predict(X_test_pca)

# Calculate and print the accuracy
accuracy = np.mean(y_pred_linear == y_test)
print("LinearSVC accuracy with PCA:", accuracy)



# pipeline = Pipeline([
#     ('scaler', StandardScaler()),
#     ('pca', PCA(n_components=27)),
#     ('svm', LinearSVC(random_state=5, dual=n_samples > n_features))
# ])

# # Fit the pipeline to the training data
# pipeline.fit(X_train, y_train)

# # Predict using the pipeline
# y_pred_linear = pipeline.predict(X_test)

# # Calculate and print accuracy
# accuracy = np.mean(y_pred_linear == y_test)
# print("Pipeline accuracy with PCA:", accuracy)

'''
poly arguments C and coef0
C - regularization parameters trade off between maximizing margin and minimizing classification error
small C - to softer margin, more points misclassified, but potentially more generalizable
large C - hard margin, wants most points correct, potentially overfits
Should be chosen through cross-validation or grid search

coef0 - the "+ b" independent term.
small coef0 - smoother decision boundary. Mitigates overfitting but doesn't capture complexity as well.
large coef0 - decision boundary more sensitive to variations in input features. Can potentially better
capture complexity but potentially overfits/doesn't generalize well

Should we try PCA? Maybe the middle columns are big determining factors? Answer = yes. Using 

Took 20 mins originally to do a GridSearch of reasonable hyperparameter variations using only 1/20th
the size of the original data. Due to exponential growth of computation needed for both higher
degree polynomials and all the pairwise comparisons that rbfs do, I knew that I needed to cut the
testing time for parameters down a bit. I used PCA to find the amount of PCAs required to retain
85% of the original information (27 PCAs). Considering the low stakes of a Connect 4 game, this
15% information loss seemed like a reasonable compromise to speed up testing times. I ran a test
on the 'poly' SVCs from degree 2-7 with all of the original data and then again using 27 principal components
instead of the 42 original components. Using PCA resulted in tests taking roughly only 75-80% of
the original amount of time while still retaining accuracies within .008 of the original data set.



When I am writing about why I chose to go with using PCA to preprocess the data, is it reasonable for me to compare PCAs directly to the original components (individual board spaces) like so:

'''

LinearSVC accuracy with PCA: 0.6570949279652655


'\npoly arguments C and coef0\nC - regularization parameters trade off between maximizing margin and minimizing classification error\nsmall C - to softer margin, more points misclassified, but potentially more generalizable\nlarge C - hard margin, wants most points correct, potentially overfits\nShould be chosen through cross-validation or grid search\n\ncoef0 - the "+ b" independent term.\nsmall coef0 - smoother decision boundary. Mitigates overfitting but doesn\'t capture complexity as well.\nlarge coef0 - decision boundary more sensitive to variations in input features. Can potentially better\ncapture complexity but potentially overfits/doesn\'t generalize well\n\nShould we try PCA? Maybe the middle columns are big determining factors? Answer = yes. Using \n\nTook 20 mins originally to do a GridSearch of reasonable hyperparameter variations using only 1/20th\nthe size of the original date. Due to exponential growth of computation needed for both higher\ndegree polynomials and all th

In [ ]:
#Bad - primarily used for binary classification problems.
accuracy_sigmoid = []
svc_sigmoid = svm.SVC(kernel='sigmoid', gamma='auto', coef0=1)
svc_sigmoid.fit(X_train, y_train)
y_pred_sigmoid = svc_sigmoid.predict(X_test)
accuracy_sigmoid.append(np.mean(y_pred_sigmoid == y_test))
print(accuracy_sigmoid)

# Evaluate the model
print("Sigmoid Kernel SVM Accuracy:", accuracy_score(y_test, y_pred_sigmoid))
print(classification_report(y_test, y_pred_sigmoid))
# Will not continue with sigmoid as it is not a good choice for this problem

In [4]:
start_time = time.time()

#Below 
# accuracies_polynomial = []

# for degree in range(2,8):
#     svc_poly = svm.SVC(kernel='poly', degree=degree, C=1.0, coef0=1)
#     svc_poly.fit(X_train, y_train)
#     y_pred_poly = svc_poly.predict(X_test)
#     accuracies_polynomial.append(np.mean(y_pred_poly == y_test))
#     elapsed_time = time.time() - start_time
#     minutes, seconds = divmod(elapsed_time, 60)
#     print(f"Accuracy for {degree} degree: ", accuracies_polynomial)
#     print(f"Elapsed time: {int(minutes)}m {seconds:.1f}s")
#     accuracies_polynomial = []





#BELOW IS PCA VERSION

accuracies_polynomial = []

for degree in range(4,5):
    svc_poly = svm.SVC(kernel='poly', degree=degree, C=1.0, coef0=1)
    svc_poly.fit(X_train_pca, y_train)
    y_pred_poly = svc_poly.predict(X_test_pca)
    accuracies_polynomial.append(np.mean(y_pred_poly == y_test))
    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"Accuracy for {degree} degree: ", accuracies_polynomial)
    print(f"Elapsed time: {int(minutes)}m {seconds:.1f}s")
    accuracies_polynomial = []

'''
Initial full scale data testing of these values showed degree of 5 was the most accurate.
'''

Accuracy for 4 degree:  [0.8211959739490823]
Elapsed time: 2m 24.5s


'\nInitial full scale data testing of these values showed degree of 5 was the most accurate.\n'

In [5]:
accuracy_rbf = []

#Gaussian Radial Basis Function
svc_rbf = svm.SVC(kernel='rbf', C=10.0, gamma=0.1)
svc_rbf.fit(X_train_pca, y_train)
y_pred_rbf = svc_rbf.predict(X_test_pca)
accuracy_rbf.append(np.mean(y_pred_rbf == y_test))
print(accuracy_rbf)

In [10]:
#BREAK BELOW WILL BE USING KFOLD 5, 10, 20 -------------------------------
#SECTION BEYOND SHOULD COMBINE KFOLD 5, 10, 20 WITH GRIDSEARCHCV - could take a very long time.

0.6485193951347796
0.6467508296314504
0.647337278106509


In [4]:
folds = [3,5]


for fold_num in folds:
    start_time = time.time()
    kf = KFold(n_splits=fold_num, shuffle=True, random_state=5)
    accuracies_polynomial = {2: [], 3: [], 4: [], 5: [], 6: [], 7: []}
    for train_index, test_index in kf.split(X_train_pca): 

        X_train_fold = X_train_pca[train_index]
        X_test_fold = X_train_pca[test_index]
        y_train_fold = y_train[train_index]
        y_test_fold = y_train[test_index]

        for degree in range(2,8):
            svc_poly = svm.SVC(kernel='poly', degree=degree, C=1.0, coef0=1)
            svc_poly.fit(X_train_fold, y_train_fold)
            y_pred_poly = svc_poly.predict(X_test_fold)
            accuracy = np.mean(y_pred_poly == y_test_fold)
            accuracies_polynomial[degree].append(accuracy)
    
    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    print(f"For {fold_num} folds:")
    print(f"Elapsed time: {int(minutes)}m {seconds:.1f}s")
    for degree in accuracies_polynomial:
        avg_accuracy = np.mean(accuracies_polynomial[degree])
        print(f"Avg accuracy for {degree} degrees: {avg_accuracy}")
        

'''
Use this section to talk about how much increase in time is taken in exchange for what
increase in accuracy for kfold. We will then use this section to determine the cv
for the GridSearchCV. Originally tried 5/10/15. Stopped it at 10 because 10 took 265%
the amount of time that 5 did and only produced a 0.41% avg increase in accuracy
across degrees 2-7 (raw increase was 0.32%). Very much not worth the time.

Increase from 3 to 5
246% more time
0.87% increase to accuracy (raw increase was 0.67%)
For such a low stakes game situation, the amount of time required for such minimal increase
in accuracy is not worth it considering how much this time will compound in a GridSearchCV
'''
        


For 3 folds:
Elapsed time: 3m 18.5s
Avg accuracy for 2 degrees: 0.7592622553471126
Avg accuracy for 3 degrees: 0.7871339537714341
Avg accuracy for 4 degrees: 0.7908136200858787
Avg accuracy for 5 degrees: 0.7789290375035754
Avg accuracy for 6 degrees: 0.7596851437568922
Avg accuracy for 7 degrees: 0.7543561874235079


KeyboardInterrupt: 

In [13]:
folds = [3,5]

for fold_num in folds:
    kf = KFold(n_splits=fold_num, shuffle=True, random_state=5)
    accuracy_rbf = []
    for train_index, test_index in kf.split(X_encoded_reduced): #CHANGE BACK TO NOT REDUCED

        X_train = X_encoded_reduced[train_index]
        X_test = X_encoded_reduced[test_index]
        y_train = y_encoded_reduced[train_index]
        y_test = y_encoded_reduced[test_index]

        #Gaussian Radial Basis Function
        svc_rbf = svm.SVC(kernel='rbf', C=1.0, gamma="auto")
        svc_rbf.fit(X_train, y_train)
        y_pred_rbf = svc_rbf.predict(X_test)
        accuracy_rbf.append(np.mean(y_pred_rbf == y_test))

    avg_accuracy = np.mean(accuracy_rbf)
    print(f"Avg accuracy with rbf with {fold_num} folds: {avg_accuracy}")

Avg accuracy with rbf with 5 folds: 0.6496971290817445
Avg accuracy with rbf with 10 folds: 0.6497032640949555
Avg accuracy with rbf with 20 folds: 0.6500035221189069


In [14]:
'''
Gamma values
High gamma - decision boundary heavily influenced by points close to it. This means the boundary
can be a lot less smooth and can capture more nuance in the data but at the risk of overfitting
Low gamma - Smoother decision boundary. Will take into account more points farther away. Can
better generalize at risk of underfitting the data with too simplistic a model.

Pros of RBF kernel
-Projects data into infinite-dimensional space - useful for complex datasets
-One less hyperparameter than poly
Cons
-too high a gamma value risks overfitting
-computationally expensive - involves all pairs of points in dataset.

Pros of poly kernel
-Interpretability?
-More configurable - could finer tune for best answer (also a con in a way)
Cons
-higher degree polynomials can have overfitting

Comparison
-if data has complex localized patterns (boundary is highly irregular), RBF may be better.
- poly strongly relies on finding correct degree, rbfs can adapt to unknown complex pattersn but at the cost
of computational power.
- both computationally expensive but the polynomial will get increasingly more expensive as degree goes higher

'''

'\nPros of RBF kernel\n-Projects data into infinite-dimensional space - useful for complex datasets\n-One less hyperparameter than poly\nCons\n-too high a gamma value risks overfitting\n-computationally expensive - involves all pairs of points in dataset.\n\nPros of poly kernel\n-Interpretability?\n-More configurable - could finer tune for best answer (also a con in a way)\nCons\n-higher degree polynomials can have overfitting\n\nComparison\n-if data has complex localized patterns (boundary is highly irregular), RBF may be better.\n- poly strongly relies on finding correct degree, rbfs can adapt to unknown complex pattersn but at the cost\nof computational power.\n- both computationally expensive but the polynomial will get increasingly more expensive as degree goes higher\n\n'

In [67]:
from sklearn.model_selection import ParameterGrid

poly_param_grid = {
    'C': [0.1, 1, 10, 100],
    'coef0': [0.0, 0.5, 1.0, 3.0, 10.0], 
    'degree': [2, 3, 4, 5, 6],
    'kernel': ['poly']
}

# grid_search = GridSearchCV(svm.SVC(), param_grid, cv=3)
# grid_search.fit(X_train_pca, y_train)
# print("Best parameters:", grid_search.best_params_)
# best_model = grid_search.best_estimator_

# y_pred_best = best_model.predict(X_test_pca)
# print("Best Kernel SVM Accuracy:", accuracy_score(y_test, y_pred_best))
# print(classification_report(y_test, y_pred_best))



poly_results = []
for params in ParameterGrid(poly_param_grid):
    model = svm.SVC(**params)
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=1)
    #line above gets the output to not blast UndefinedMetricWarnings.
    poly_results.append({
        'params': params,
        'accuracy': accuracy,
        'report': report
    })

# Sort results by accuracy in descending order
poly_results.sort(key=lambda x: x['accuracy'], reverse=True)

# Optionally, print results
first_result = True
index = 1
for result in poly_results:
    if first_result:
        print(f"Best parameters: {result['params']}")
        print(f"Best Kernel SVM Accuracy: {result['accuracy']:.3f}")
        first_result = False
        index += 1
    else:
        print(f"The {index}nd/th best option")
        print(result['params'])
        print(f"{result['accuracy']:.3f}")
        index += 1




'''
Ran the code above with 1/20th the total data set and it took 20 minutes. Considering the
exponential increase in time it would take to process, needed to choose an efficient path
for testing. Initial param grids were set up dumb - lots of redundant tests - Using GridSearchCV
with cv=3 which greatly increases time as well. Realizes from data that a 3 kfold actually slightly
reduced accuracy in 1/2 data 24 PCs run so switched from GridSearchCV to simply ParameterGrid.

1/20th data 24 PCs no CV - All degree 7s in the last half of 120 combinations. As seen in note in Excel,
smaller data sets typically favor smaller degrees but given 7 was still reasonably lower than even 6
in the original data run and how badly it performed here, I felt safe in removing it as it nearly
doubles the time required.

Data below is using 1/2 the full data set with 24 PCs AND NO CROSS VALIDATION! Did not finish in 180 mins.
'''

In [59]:
rbf_param_grid = {
    'C': [1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1], 
    'kernel': ['rbf']
}

rbf_results = []

for params in ParameterGrid(rbf_param_grid):
    model = svm.SVC(**params)
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=1)
    #line above gets the output to not blast UndefinedMetricWarnings.
    rbf_results.append({
        'params': params,
        'accuracy': accuracy,
        'report': report
    })

# Sort results by accuracy in descending order
rbf_results.sort(key=lambda x: x['accuracy'], reverse=True)

# Optionally, print results
first_result = True
index = 1
for result in rbf_results:
    if first_result:
        print(f"Best parameters: {result['params']}")
        print(f"Best Kernel SVM Accuracy: {result['accuracy']:.3f}")
        first_result = False
        index += 1
    else:
        print(f"The {index}nd/th best option")
        print(result['params'])
        print(f"{result['accuracy']:.3f}")
        index += 1

'''
Ran 1/2 data 24 PCs - In the top half of results (10 of 20), gamma of .5 shows up once at 10th place.
Removed gamma .5 for speed in full data set search.  C val of .5 only present in places 13 and beyond.
Removed C val of .5 for speed.

Data below is using the full data set no PCs BUT NO CROSS VALIDATION!
'''

Best parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best Kernel SVM Accuracy: 0.831
The 2nd/th best option
{'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
0.829
The 3nd/th best option
{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.826
The 4nd/th best option
{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
0.821
The 5nd/th best option
{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
0.816
The 6nd/th best option
{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.804
The 7nd/th best option
{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
0.801
The 8nd/th best option
{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.801
The 9nd/th best option
{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.791
The 10nd/th best option
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.771
The 11nd/th best option
{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
0.748
The 12nd/th best option
{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.684


'\nRan 1/2 data 24 PCs - In the top half of results (10 of 20), gamma of .5 shows up once at 10th place.\nRemoved gamma .5 for speed in full data set search.  C val of .5 only present in places 13 and beyond.\nRemoved C val of .5 for speed.\n'

In [21]:
n_samples_all, n_features_all = X_train_pca.shape
auto = 1 / n_features_all
print(f"Auto: {auto}")
scale = 1 / (n_features_all * np.var(X_train_pca, axis=0))
print(f"Scale: {scale}")
mean_scale_gamma = np.mean(scale)
print("Mean Scale Gamma:", mean_scale_gamma)

Auto: 0.041666666666666664
Scale: [0.0313389  0.03276203 0.03670258 0.03692228 0.03734812 0.043479
 0.08751912 0.09102104 0.09625117 0.10039322 0.11197217 0.12027572
 0.12680769 0.14474259 0.15691405 0.16041014 0.19386078 0.21602958
 0.23085219 0.28391465 0.28816793 0.32670557 0.35266103 0.45419528]
Mean Scale Gamma: 0.15671861840380516
